In [1]:
import pandas as pd
from pandarallel import pandarallel

In [2]:
df_trainset = pd.read_csv("data/04_train_set.tsv", sep="\t")
df_trainset

,chr,start,end,strand,class
0,chr1,12227,12612,+,1
1,chr1,12721,13220,+,1
2,chr1,12057,12178,+,1
3,chr1,12697,12974,+,1
4,chr1,13052,13220,+,1
...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0
519030,chrY,25513173,25513588,-,0
519031,chrY,25513745,25516715,-,0
519032,chrY,25525288,25527646,-,0


In [7]:
def matchrc(row):
    potential_matches = !tabix data/resources/recount3.bed.gz {row.chr}:{row.start}-{row.end} | cut -f 2,3,5

    potential_matches = [
        int(entry.split('\t')[2])
        for entry in potential_matches
        if row['start'] == int(entry.split('\t')[0]) and row['end'] == int(entry.split('\t')[1]) #["RC3-Score"]
    ]

    if potential_matches:
        return potential_matches[0]
    else:
        return 0

In [8]:
pandarallel.initialize(nb_workers=12)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [9]:
%%time
df_test = df_trainset.copy().head(10)
df_test["RC3"] = df_test.apply(matchrc, axis=1)
df_test

CPU times: user 6.87 ms, sys: 11.4 ms, total: 18.3 ms
Wall time: 100 ms


,chr,start,end,strand,class,RC3
0,chr1,12227,12612,+,1,59151
1,chr1,12721,13220,+,1,61021
2,chr1,12057,12178,+,1,0
3,chr1,12697,12974,+,1,21
4,chr1,13052,13220,+,1,526
5,chr1,13374,13452,+,1,0
6,chr1,14501,15004,-,1,0
7,chr1,15038,15795,-,1,7437205
8,chr1,15947,16606,-,1,2667727
9,chr1,16765,16857,-,1,7006744


In [10]:
%%time
df_trainset["recount3_score"] = df_trainset.parallel_apply(matchrc, axis=1)
df_trainset

CPU times: user 25.3 ms, sys: 343 ms, total: 368 ms
Wall time: 5min 24s


,chr,start,end,strand,class,recount3_score
0,chr1,12227,12612,+,1,59151
1,chr1,12721,13220,+,1,61021
2,chr1,12057,12178,+,1,0
3,chr1,12697,12974,+,1,21
4,chr1,13052,13220,+,1,526
...,...,...,...,...,...,...
519029,chrY,25464577,25465486,+,0,0
519030,chrY,25513173,25513588,-,0,31374
519031,chrY,25513745,25516715,-,0,67
519032,chrY,25525288,25527646,-,0,18958


In [11]:
# Optional backup:
df_trainset.to_csv("data/05_trainset_recount3_feature.tsv", sep="\t", index=False)

In [12]:
! sha512sum data/05_trainset_recount3_feature.tsv

2edc6e69902481e6fb62fb03ea084cd35baba20583c62e9a459d9656010c4591ceef9d0af187359f967ecad876ad58b71521bdf891fb60e4e7ee5b0704a5ca2c  data/05_trainset_recount3_feature.tsv
